In [1]:
%reset
%env HSA_OVERRIDE_GFX_VERSION=10.3.0

env: HSA_OVERRIDE_GFX_VERSION=10.3.0


In [2]:
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchinfo import summary
from Classes import StockData, LSTM

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

RawStockDataDirectory = 'StockData/Raw/'
ProcessedStockDataDirectory = 'StockData/Processed/'
DatasetsDirectory = 'StockData/Datasets/'

inputLength = 90
outputLength = 14

In [3]:
torch.cuda.is_available()
device = torch.device('cuda')

In [4]:
dataset = torch.load(DatasetsDirectory + 'MasterDataset10.pt')

In [5]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [46]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [47]:
model = LSTM(22, 128, 3, 1)
model.to(device)

LSTM(
  (lstm): LSTM(22, 128, num_layers=3, dropout=0.2)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [48]:
summary(model)

Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              342,016
├─Linear: 1-2                            129
├─Dropout: 1-3                           --
Total params: 342,145
Trainable params: 342,145
Non-trainable params: 0

In [49]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [53]:
num_epochs = 30
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

/home/phill/Documents/MachineLearning/StockPricePrediction/VirtualEnvirorment/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32, 14, 1])) that is different to the input size (torch.Size([90, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (90) must match the size of tensor b (14) at non-singleton dimension 1